## Simulation of a M/M/1 queue using processes

This simulation is adapted from the Bank Renege example in the documentation of a previous version of SimJulia: http://simjuliajl.readthedocs.io/en/stable/examples/1_bank_renege.html

In [1]:
using SimJulia
using Distributions
using ResumableFunctions
using RDST, Random

Let's first simulate a fixed number of clients.

In [2]:
const RANDOM_SEED = 200
const NEW_CUSTOMERS = 5  # Total number of customers
const INTERVAL_CUSTOMERS = 2.0  # Generate new customers roughly every x seconds
const MEAN_SERVICE = 1.99

# The macro @resumable allows to suspend a function until some event wakes it up.
@resumable function source(env::Simulation, number::Int, interval::Float64, counter::Resource)
    d = Exponential(interval)
    for i in 1:number
        # The customer service time is random. During the service, the counter
        # is not available to any other customer.
        @yield timeout(env, rand(d))
        @process customer(env, i, counter, MEAN_SERVICE)
    end
end

@resumable function customer(env::Simulation, idx::Int, counter::Resource, time_in_system::Float64)
    # Record the arrival time in the system
    arrive = now(env)
    println("$arrive: arrival of customer $idx")
    @yield request(counter)
    # The simulation clock now contains the time when the client goes to the server.
    wait = now(env) - arrive
    # Record the waiting time
    waits[idx] = wait
    println("$(now(env)): customer $idx has waited $wait")
    @yield timeout(env, rand(Exponential(time_in_system)))
    println("$(now(env)): customer $idx: finished")
    @yield release(counter)
end

customer (generic function with 1 method)

In [3]:
# Setup and start the simulation
println("M/M/1 with processes")
waits = zeros(NEW_CUSTOMERS)

M/M/1 with processes


5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [4]:
Random.seed!(RANDOM_SEED)
env = Simulation()

Simulation time: 0.0 active_process: nothing

In [5]:
# Start processes and run
counter = Resource(env, 1)
@process source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)

Process 1

In [6]:
run(env)

1.2050403125807367: arrival of customer 1
1.2050403125807367: customer 1 has waited 0.0
1.4181503510677775: arrival of customer 2
1.7579446126823792: arrival of customer 3
3.3655840436959545: arrival of customer 4
3.70641375839329: customer 1: finished
3.70641375839329: customer 2 has waited 2.2882634073255126
3.7636956918681514: customer 2: finished
3.7636956918681514: customer 3 has waited 2.0057510791857722
4.581834601908197: customer 3: finished
4.581834601908197: customer 4 has waited 1.2162505582122423
6.231843986748496: arrival of customer 5
11.58459018900139: customer 4: finished
11.58459018900139: customer 5 has waited 5.352746202252893
15.538707845736354: customer 5: finished


We can compute the mean waiting time by

In [7]:
mean(waits)

2.172602249395284

However, most of the time, we do not know the number of clients. We first set the end of simulation event by specifying an time horizon when running the simulation.

In [8]:
@process source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)
run(env, 5.0)

We observe however that the simulation time has not been reset to 0. A simple solution is to create a new simulation environment. This also requires to set the resource again.

In [9]:
env = Simulation()
counter = Resource(env, 1)
@process source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)
run(env, 5.0)

1.2559678592926755: arrival of customer 1
1.2559678592926755: customer 1 has waited 0.0
2.781482657038433: arrival of customer 2
3.6650526534802976: arrival of customer 3


The random draws are different but we can produce the same as previously by using the same seed, i.e. the same initial state.

In [10]:
Random.seed!(RANDOM_SEED)
env = Simulation()
counter = Resource(env, 1)
@process source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter)
run(env, 10.0)

1.2050403125807367: arrival of customer 1
1.2050403125807367: customer 1 has waited 0.0
1.4181503510677775: arrival of customer 2
1.7579446126823792: arrival of customer 3
3.3655840436959545: arrival of customer 4
3.70641375839329: customer 1: finished
3.70641375839329: customer 2 has waited 2.2882634073255126
3.7636956918681514: customer 2: finished
3.7636956918681514: customer 3 has waited 2.0057510791857722
4.581834601908197: customer 3: finished
4.581834601908197: customer 4 has waited 1.2162505582122423
6.231843986748496: arrival of customer 5


However, a possible issue is that a customer never finishes his service. If we want to ensure that the customer complete his journey in the system, we have to modify the source function. We can circumvent it by redefining the source function so that no customer is generated after a horizon limit, but we do not put a limit when calling the run function.

In [11]:
@resumable function source2!(env::Simulation, number::Int, interval::Float64, counter::Resource, limit::Float64, nserved::Array{Int64,1})
    nserved[1] = 0
    d = Exponential(interval)
    i = 0
    while true
        i += 1
        @yield timeout(env, rand(d))
        if (now(env) > limit) break end
        @process customer(env, i, counter, MEAN_SERVICE)
        nserved[1] += 1
    end
end

source2! (generic function with 1 method)

In [12]:
nserved = [ 0 ]
waits = zeros(100)
Random.seed!(RANDOM_SEED)
env = Simulation()
counter = Resource(env, 1)

@process source2!(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter, 100.0, nserved)
run(env)

1.2050403125807367: arrival of customer 1
1.2050403125807367: customer 1 has waited 0.0
1.4181503510677775: arrival of customer 2
1.7579446126823792: arrival of customer 3
3.3655840436959545: arrival of customer 4
3.70641375839329: customer 1: finished
3.70641375839329: customer 2 has waited 2.2882634073255126
3.7636956918681514: customer 2: finished
3.7636956918681514: customer 3 has waited 2.0057510791857722
4.581834601908197: customer 3: finished
4.581834601908197: customer 4 has waited 1.2162505582122423
6.231843986748496: arrival of customer 5
10.205831581457005: arrival of customer 6
11.58459018900139: customer 4: finished
11.58459018900139: customer 5 has waited 5.352746202252893
12.065706907215944: arrival of customer 7
12.8342782089976: customer 5: finished
12.8342782089976: customer 6 has waited 2.6284466275405958
13.5912217049617: arrival of customer 8
14.474791701403564: arrival of customer 9
18.035640107194286: arrival of customer 10
18.63582099618231: customer 6: finished

This raises the question: should we flush the entities in the system at the end of the horizon of allow the entities in the system to complete their process? It depends on the context!

In our case, the mean waiting time is

In [13]:
mean(waits[10:nserved[1]])

9.250166221866102

In [14]:
mean(waits[1:nserved[1]])

8.37958449066649

In [15]:
mean(waits[20:nserved[1]])

10.265702791470435

In [16]:
mean(waits[50:nserved[1]])

16.972298792366896

In [17]:
@resumable function new_source!(env::Simulation, interval::Float64, counter::Resource, limit::Float64, nserved::Array{Int64,1})
    nserved[1] = 0
    i = 0
    d = Exponential(interval)
    while (true)
        @yield timeout(env, rand(d))
        if (now(env) > limit) break end
        i += 1
        @process new_customer(env, i, counter, MEAN_SERVICE, new_waits)
    end
    nserved[1] = i
 end

new_source! (generic function with 1 method)

In [18]:
@resumable function new_customer(env::Simulation, idx::Int, counter::Resource, time_in_system::Float64, waits::Array{Float64,1})
    # Record the arrival time in the system
    arrive = now(env)
    println("$arrive: arrival of customer $idx")
    @yield request(counter)
    # The simulation clock now contains the time when the client goes to the server.
    wait = now(env) - arrive
    # Record the waiting time
    waits = push!(waits, wait)
    println("$(now(env)): customer $idx has waited $wait")
    @yield timeout(env, rand(Exponential(time_in_system)))
    println("$(now(env)): customer $idx: finished")
    @yield release(counter)
end

new_customer (generic function with 1 method)

In [19]:
nserved = [ 0 ]
Random.seed!(RANDOM_SEED)
env = Simulation()
counter = Resource(env, 1)
new_waits = Float64[]

@process new_source!(env, INTERVAL_CUSTOMERS, counter, 5.0, nserved)

run(env)

1.2050403125807367: arrival of customer 1
1.2050403125807367: customer 1 has waited 0.0
1.4181503510677775: arrival of customer 2
1.7579446126823792: arrival of customer 3
3.3655840436959545: arrival of customer 4
3.70641375839329: customer 1: finished
3.70641375839329: customer 2 has waited 2.2882634073255126
3.7636956918681514: customer 2: finished
3.7636956918681514: customer 3 has waited 2.0057510791857722
4.581834601908197: customer 3: finished
4.581834601908197: customer 4 has waited 1.2162505582122423
11.58459018900139: customer 4: finished


In [20]:
new_waits

4-element Vector{Float64}:
 0.0
 2.2882634073255126
 2.0057510791857722
 1.2162505582122423

In [21]:
nserved[1]

4